In [4]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import pandas as pd
from typing import *

PyTorch Version:  1.12.1
Torchvision Version:  0.13.1a0


In [7]:
path_to_train_data = "./data/annotations/train.pkl"
path_to_val_data = "./data/annotations/val.pkl"

In [6]:
def load_data(path: str) -> pd.DataFrame:
    f = open(path, 'r')
    data: List[Tuple[str, int, int, int]] = []
    for line in f.readlines():
        split_line: List[str] = line.split(' ')

        data.append((split_line[0], int(split_line[1]), int(split_line[2]), int(split_line[3])))

    df = pd.DataFrame(data).rename(columns={0: 'file_name', 1: 'id', 2: 'species', 3: 'breed'})
    return df

In [ ]:
def label_to_one_hot(df: pd.DataFrame) -> pd.DataFrame:
    breed = df['breed']

    max_label: int = np.max(breed)
    one_hot_labels: List[np.array] = []

    for elem in breed:
        one_hot = np.zeros(max_label, dtype=int)
        one_hot[elem-1] = 1

        one_hot_labels.append(one_hot)

    df['breed_onehot'] = one_hot_labels

    return df

### Inputs

In [ ]:
model_name = "resnet"
batch_size = 8
num_epochs = 15

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs = 25):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epcoh in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()
        running_loss = 0.0
        for inputs, labels in dataloaders[phase]
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == "train"):

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history






In [ ]:
def set_parameter_requires_grad(model):
    for param in model.paramters():
        param.requires_grad = False

Initialize Model

In [ ]:
model.fc = nn.Linear(512, num_classes)

In [ ]:
def initialize_model(num_classes, use_pretrained = True):
    model_ft = None
    input_size = 0

    model_ft = models.resnet18(use_pretrained=use_pretrained)
    set_parameters_requires_grad(model_ft)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    return model_ft, input_size